In [2]:
import urllib2
import urllib
from bs4 import BeautifulSoup
import sys
import requests
import json
import time
import re
import csv
import re
import time
import pandas as pd
import numpy as np
from pandas import ExcelWriter

c:\python27\lib\site-packages\cffi\model.py:526: UserWarning: 'point_conversion_form_t' has no values explicitly defined; next version will refuse to guess which integer type it is meant to be (unsigned/signed, int/long)
  % self._get_c_name())


In [3]:
def post_(url,params1):
	r = requests.post(url,params=params1)
	return r
def get_(url,params1):
	r = requests.get(url,params=params1)
	return r
def get_nop(url,params1):
	r = requests.get(url)
	return r
def get_prod_details(url):
	html = get_nop(url,{})
	soup = BeautifulSoup(html.text,"html.parser")
	#title-wrap line fk-font-family-museo section omniture-field
	prod ={}
	name = soup.findAll("div", { "class" : "title-wrap line fk-font-family-museo section omniture-field" })
	prod['name']=name[0].text.strip()
	#keyFeatures specSection
	description = soup.findAll("div", { "class" : "gd-col gu12"})
	text = ''
	for d in description:
		text = text + get_text(d).strip()
	prod['description'] = my_strip(text)
	#key-specifications fk-ul-disc lpadding20 line fk-font-11 fk-fontlight
	description2 = soup.findAll("ul", { "class" : "key-specifications fk-ul-disc lpadding20 line fk-font-11 fk-fontlight" })
	text = ''
	for d in description2:
		text = text + get_text(d)
	prod['description2'] = my_strip(text)
	return prod
def parse_prod(prod):
	#print prod
	#fk-display-block
	title = prod.findAll("a", { "class" : "fk-display-block" },href=True)
	p={}
	p['url'] = base_url + title[0]['href']
	return p 
	
def get_text(node):
	#print node
	addinfo = node
	if len(addinfo) > 0:
			addinfo = ''.join(addinfo.findAll(text=True))
	else:
		addinfo = ''
	return addinfo
def my_strip(text):
	text = re.sub('\s+',' ',text)
	return text

In [29]:
api_url = 'https://en.wikipedia.org/wiki/List_of_government_schemes_in_India';

In [30]:
for i in range(0,1):
    params = {
    'sid':"mgl,u35",
    'filterNone':True,
    'start':1,
    'ajax':True,
    '_':1453012261736
    };
for i in range(1,2):
    p = params
    p['start'] = i
    html = get_(api_url,p)
    soup = BeautifulSoup(html.text,"html.parser")

In [31]:
all_data = soup.find("table", { "class" : "wikitable sortable" }).find_all('tr')

In [67]:
data_ = []
data_cols = []
for each_data in all_data:
    tds = each_data.find_all('td')
    if(len(tds)>0):
        tds = [x.text.strip().encode('ascii','ignore') for x in tds]
        tds[2] = tds[2][23:]
        data_.append(tds)
    if(len(tds)==0):
        ths = each_data.find_all('th')
        if(len(ths)>0):
            data_cols = [x.text.strip().encode('ascii','ignore') for x in ths]

In [69]:
data_ = pd.DataFrame(data_, columns = data_cols)

In [70]:
data_.head(5)

,Scheme,Ministry,Date of Launch,Outlay/Status,Sector,Provisions
0,Atal Pension Yojana [1],,"May 9, 2015",,Pension,Social Sector Scheme pertaining to Pension Sector
1,Bachat Lamp Yojana,MoP,2009,,Electrification,reduce the cost of compact fluorescent lamps
2,Central Government Health Scheme,MoHFW,1954,,Health,comprehensive medical care facilities to Centr...
3,Deendayal Disabled Rehabilitation Scheme,MoSJE,2003,,Social Justice,Create an enabling environment to ensure equal...
4,Deen Dayal Upadhyaya Gram Jyoti Yojana [2],MoP,2015,,Rural Power Supply,It is a Government of India program aimed at p...


In [71]:
data_.to_csv('scraped_data_wiki.csv')